## Proyecto Fundamentos de Ciencia de Datos

In [ ]:
# librerias

# Base de datos
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# K medias 
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
import plotly.express as px
from sklearn.metrics import silhouette_score
# predicción
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
# guardar los modelos
import joblib


In [ ]:
# Base de datos
accounts=pd.read_csv('archive/accounts.csv')
products=pd.read_csv('archive/products.csv')
sales=pd.read_csv('archive/sales_pipeline.csv')
team=pd.read_csv('archive/sales_teams.csv')
done_deals=sales[sales['deal_stage']=='Won']

In [ ]:
# Limpieza
df = sales.merge(team, on='sales_agent') 
df = df[~df['account'].isna()]  
df = df.merge(accounts,on='account') 
df['subsidiary_of']  = df['subsidiary_of'].fillna('')
df['product']=df['product'].str.replace('GTXPro','GTX Pro')

EDA

Análisis de una variable

In [ ]:
#analisis de ventas (en dataframe sale):

#close_value: distribucion de ventas cerradas por valor de cierre (solo done deals) ((histograma con 12 bins))
counts, bin_edges = np.histogram(done_deals['close_value'], bins=12)
bin_labels = [f"{round(start)} - {round(end)}" for start, end in zip(bin_edges[:-1], bin_edges[1:])]
hist_df_canvas = pd.DataFrame({ 'bin': bin_labels, 'count': counts})
hist_df_canvas.to_csv('1v_close_value.csv',index=False)
#deal_stage: distribucion de las oportunidades ((pie chart))
sales['deal_stage'].value_counts().to_csv('1v_deal_stage.csv')
#sales_agent: distribucion de ventas cerradas por agente (solo done deals) #top 10 agents en vol de ventas ((bar chart))
done_deals['sales_agent'].value_counts()[:10].to_csv('1v_sales_agent.csv')
#product: distribucion de ventas cerradas por producto (solo done deals) ((horizontal chart))
done_deals['product'].value_counts().to_csv('1v_product.csv')
#account: distribucion de clientes con los que se hizo negocio (solo done deals) #top 10 clients en vol de ventas ((bar chart))
done_deals['account'].value_counts()[:10].to_csv('1v_account.csv')
#close_date: distribucion de ventas cerradas por fecha (solo done deals) ((line chart))
done_deals['close_date'].str[:7].value_counts().to_csv('1v_close_date.csv')

In [ ]:
# analisis de clientes (en dataframe accounts):
accounts[['account','revenue','employees']].to_csv('1v_histograms.csv',index=False)

Análisis de dos variables

In [ ]:
#sobre ventas  

# distribucion del close_value para ventas cerradas por agente y producto 
done_deals1=done_deals.copy()
done_deals1.loc[:,'product'] = done_deals1['product'].astype('category').cat.codes
done_deals1.loc[:,'sales_agent'] = done_deals1['sales_agent'].astype('category').cat.codes
g=sns.pairplot(done_deals1[['sales_agent','close_value','product']],
             plot_kws={"color": "#4283E6F6", "edgecolor": "none"},     
            diag_kws={"color": "#37BA6BF6", "edgecolor": "gray"},)
sns.set(style="white", rc={"axes.facecolor": "#f2f1ec", "figure.facecolor": "#f2f1ec"})
plt.show()

# sobre ventas y empresas

# distribucion de close_value por sector economico, employees y revenue
df1=df.copy()
df1=df1[df1['deal_stage']=='Won']  
df1.loc[:,'sector'] = df1['sector'].astype('category').cat.codes
g=sns.pairplot(df1[['sector', 'revenue','close_value']],
             plot_kws={"color": "#558DC2F6", "edgecolor": "none"},     
            diag_kws={"color": "#8A1DACF6", "edgecolor": "gray"} ) 

# Ajustar cada eje
# for ax in g.axes.flatten():
#     if ax is not None:
#         ax.set_facecolor('#F2F1EC')  # fondo del subgráfico
#         for spine in ax.spines.values():
#             spine.set_visible(False)  # quitar bordes
#         ax.set_xlabel("")            # quitar etiquetas
#         ax.set_ylabel("")
#         ax.set_xticks([])           # quitar ticks
#         ax.set_yticks([])
plt.show()


In [ ]:
# Planteamiento del problema
# ¿Qué meta-data se puede generar? aqui marcar los objetivos, la hoja de ruta del trabajo (basicamente lo que hicimos con Marco)
# Se quiere evaluar el desempeño de los agentes de ventas, de modo que, dada la informacion de un nuevo cliente podamos asignar al agente que le atienda de manera óptima:
# en terminos de: monto más alto de la venta, mayor experiencia del agente con ese tipo de clientes, etc. Por esto, tambien se busca clasificar a los clientes en clusters de acuerdo a sus características

In [ ]:
# preprocesamiento de datos

df['engage_date']=pd.to_datetime(df['engage_date'])
df['close_date']=pd.to_datetime(df['close_date'])
df['close_value'] = df['close_value'].fillna(0)   
df=df.merge(products,how='left',on='product')

In [ ]:
# Implementar un modelo de clasificación o segmentacion de agentes (regresion logistica, k-medias) 

# kmeans (mejor parámetro)
categorical_cols = ['sector', 'office_location', 'subsidiary_of']
numeric_features = ['year_established', 'revenue', 'employees']
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_cols)
    ])
X_processed = preprocessor.fit_transform(accounts) 
k_values = range(2, 11)
inertia = []
silhouette_scores = []
for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_processed)
    inertia.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(X_processed, kmeans.labels_))
plt.figure(figsize=(12, 5))

# Método del codo
plt.subplot(1, 2, 1)
plt.plot(k_values, inertia, marker='o')
plt.title('Método del Codo')
plt.xlabel('Número de Clústers')
plt.ylabel('Inercia')

# Índice de silueta
plt.subplot(1, 2, 2)
plt.plot(k_values, silhouette_scores, marker='o')
plt.title('Índice de Silueta')
plt.xlabel('Número de Clústers')
plt.ylabel('Score de Silueta')

plt.tight_layout()
plt.show()

In [ ]:
# K medias en la tabla de clientes

categorical_cols = ['sector', 'office_location', ] 
numeric_features=['year_established','revenue', 'employees'] 
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_cols)
    ])
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('kmeans', KMeans(n_clusters=4, random_state=42))
])
pipeline.fit(accounts)
accounts['Clúster'] = pipeline.named_steps['kmeans'].labels_ 
sns.set(style="white", rc={"axes.facecolor": "#f2f1ec", "figure.facecolor": "#f2f1ec"})
g = sns.pairplot(
    accounts,
    hue='Clúster',          
    palette='tab10',        
    diag_kind='kde',        
)
for ax in g.axes.flatten():
    if ax is not None:
        ax.set_facecolor('#F2F1EC')  
        for spine in ax.spines.values():
            spine.set_visible(False)  
        ax.set_xlabel("")            
        ax.set_ylabel("")
        ax.set_xticks([])           
        ax.set_yticks([])
    
accounts['Clúster'].value_counts().to_csv('kmeans.csv')
joblib.dump(pipeline, 'kmeans_pipeline.pkl')

In [ ]:
# grafica con PCA
 
features_transformed = pipeline.named_steps['preprocessor'].transform(accounts)
pca = PCA(n_components=2)
components = pca.fit_transform(features_transformed)
plot_df = pd.DataFrame(components, columns=['CP1', 'CP2'])
plot_df['cluster'] = accounts['Clúster'].astype(str)
plot_df['account'] = accounts['account']
plot_df['revenue'] = accounts['revenue']
plot_df['employees'] = accounts['employees']
fig = px.scatter(
    plot_df, x='CP1', y='CP2', color='cluster',
    labels={'cluster': 'Clúster'},)
fig.update_layout(
    plot_bgcolor='#f2f1ec',
    paper_bgcolor='#f2f1ec',
    legend_title_text='Clúster',
)
fig.update_layout(width=500, height=500)
fig.show()
 

In [ ]:
# implementar un modelo de predicción (xgboost, random forest, regresion lineal) para predecir performance futuro
# Dividir los datos en conjuntos de entrenamiento y prueba.
# realizar validacion cruzada

In [ ]:
# preprocesamiento de datos para el modelo

df=df[df['deal_stage']=='Won']
df=df.merge(accounts[['account','Clúster']], on='account', how='left')
model_df=df[[ 'sales_agent', 'product', 'engage_date', 'close_date', 'close_value', 'manager', 'regional_office', 'series', 'sales_price', 'Clúster']]

In [ ]:
# Probar diferentes modelos

model_df=df[[ 'sales_agent', 'product',  'engage_date', 'close_date', 'close_value', 'manager', 'regional_office', 'series', 'sales_price', 'Clúster',  ]]
categorical_features = ['sales_agent', 'product', 'manager', 'regional_office', 'series', 'Clúster']
numerical_features = ['sales_price']

X = model_df[categorical_features + numerical_features]
y = model_df['close_value']
preprocessor = ColumnTransformer([
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
    ('num', SimpleImputer(strategy='mean'), numerical_features)
])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
models = {
    'RandomForest': GridSearchCV(
        Pipeline([('preprocessor', preprocessor), ('model', RandomForestRegressor())]),
        param_grid={'model__n_estimators': [100, 200], 'model__max_depth': [None, 10, 20]},
        cv=5
    ),
    'XGBoost': GridSearchCV(
        Pipeline([('preprocessor', preprocessor), ('model', XGBRegressor(objective='reg:squarederror'))]),
        param_grid={'model__n_estimators': [100, 200], 'model__max_depth': [3, 6]},
        cv=5
    ),
    'LinearRegression': GridSearchCV(
        Pipeline([('preprocessor', preprocessor), ('model', LinearRegression())]),
        param_grid={},
        cv=5
    )
}
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    results[name] = {
        'Best Params': model.best_params_,
        'MSE': mse,
        'R2': r2
    }
for model_name, metrics in results.items():
    print(f"Modelo: {model_name}")
    print(f"  Mejores parámetros: {metrics['Best Params']}")
    print(f"  Error cuadrático medio (MSE): {metrics['MSE']:.2f}")
    print(f"  Coeficiente de determinación (R2): {metrics['R2']:.2f}\n")


In [ ]:
# Modelo Random Forest con mejores parámetros 

rf_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42))
])
rf_pipeline.fit(X_train, y_train)
joblib.dump(rf_pipeline, 'random_forest_pipeline.pkl')

# Importancia de variables
feature_names = rf_pipeline.named_steps['preprocessor'].transformers_[0][1].get_feature_names_out(categorical_features).tolist()
feature_names += numerical_features
importances = rf_pipeline.named_steps['model'].feature_importances_

importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

plt.figure(figsize=(5, 5))
plt.barh(importance_df['Feature'][:8], importance_df['Importance'][:8])
plt.xlabel('Importancia')
plt.title('Importancia de las variables - Random Forest')
plt.gca().invert_yaxis()
plt.tight_layout()
# plt.savefig('feature_importance_rf.png')
plt.show()
# Predicciones
y_pred = rf_pipeline.predict(X_test)
comparison_df = pd.DataFrame({'Real': y_test.values, 'Predicho': y_pred})
comparison_df[:201].to_csv('predicciones_rf.csv',index=False)

In [ ]:
# Conclusión
# responder al problema de a quién le doy el incentivo